# Email Metric Generation

In [1]:
import pandas as pd
import numpy as np

from query_gen import query_gen

### Part 1: Change retailer_id to the retailer of interest. A lookup table for retailer Id's is below:

In [2]:
retailer_id = 4
qg = query_gen(retailer_id)

##### ID Lookup table

In [3]:
id_lookup = qg.run_retailer_id_query()

In [55]:
id_lookup.head()

,id,name
0,1,King Soopers
1,2,Safeway
2,3,Target
3,4,Walmart
4,5,Walgreens


In [60]:
print 'Retailer: ID \n'
for i, row in id_lookup.head(10).iterrows():
    print '{0}: {1}'.format(row['name'], row['id'])

Retailer: ID 

King Soopers: 1
Safeway: 2
Target: 3
Walmart: 4
Walgreens: 5
Duane Reade: 6
Kroger: 7
Ralphs: 8
Publix: 9
CVS Pharmacy: 10


### Part 2: Run SQL Queries, you must rerun Part 1 to change retailer

In [21]:
df_cat = qg.run_categories_query()
df_comp = qg.run_competitor_query()
df_index = qg.run_index_analysis_query()

### Part 3: Calculated where retailer over/under indexed 

In [43]:
interest = df_index.query('interest_flag == 0')
others = df_index.query('interest_flag == 1')
df_ind_mg = interest.merge(others, on='cat_name')

interest_total = np.sum(df_ind_mg.n_receipt_items_y)
others_total = np.sum(df_ind_mg.n_receipt_items_x)

df_ind_mg['pct_interest'] = df_ind_mg.eval('n_receipt_items_y / @interest_total')
df_ind_mg['pct_other'] = df_ind_mg.eval('n_receipt_items_x / @others_total')
df_ind_mg['interest_indexed'] = df_ind_mg.eval('pct_interest / pct_other')

df_ind_mg.sort_values('interest_indexed', inplace=True, ascending=False);

### Part 4: Print Results

###### Change n_rows to see more or less competitors

In [48]:
n_rows = 3

for index, row in df_comp.head(n_rows+1).iterrows():
    if index == 0:
        retailer_name = row['name']
        print "Top {0} competitors for {1}'s {2:,} light customers"\
        .format(n_rows, retailer_name, row['n_customers'])
        print '\n'
    else:
        cats = list(df_cat[df_cat.retailer_name == row['name']].cat_name[:3].values)
        print '{0} has {1:,} light {2} customers.'.format(row['name'], row['n_customers'], retailer_name)
        print 'On average they spend {0:.2f} on {1:,} items per trip and take {2:.2f} trips per month.'\
        .format(row['avg_tot_per_trip'], row['avg_items_per_trip'], row['avg_trips_month'])
        print 'Customers most often purchased in the categories {0}, {1}, and {2}.'\
        .format(*cats)
        print '\n'

top_index = list(df_ind_mg.head(3).cat_name.values)
bottom_index = list(df_ind_mg.tail(3).cat_name.values)
        
tmp = top_index+bottom_index
    
print '''{0} over indexed in the categories of {1}, {2}, and {3} 
and under indexed in the categories of {4}, {5}, and {6}.
'''.format(retailer_name, *tmp)

Top 3 competitors for Walmart's 187,640 light customers


Target has 39,578 light Walmart customers.
On average they spend 59.49 on 16 items per trip and take 0.84 trips per month.
Customers most often purchased in the categories Snacks, Cookies, & Chips, Household Essentials, and Beverages.


Kroger has 23,721 light Walmart customers.
On average they spend 74.45 on 52 items per trip and take 0.91 trips per month.
Customers most often purchased in the categories Produce, Dairy, and Beverages.


Publix has 18,810 light Walmart customers.
On average they spend 61.64 on 21 items per trip and take 1.18 trips per month.
Customers most often purchased in the categories Produce, Dairy, and Beverages.


Walmart over indexed in the categories of Sporting Goods, Craft Supplies, and Automotive 
and under indexed in the categories of Wine, Cigarettes & Tobacco, and Gift Cards.

